## Demonstration of Name-Based Access Control and Group Keys

Uses Python scripts on FABRIC nodes to exchange files and perform key exchanges based upon NDN's NAC designs.

While it does not run on an active NDN-DPDK node like intended in FABRIC_NAC, it demonstrates the key exchange in accordance with the separation of producer and data owner nodes.

See attached report for image sources.

NOTE: if you run parts of this notebook (and it is working) and come back to your slice after your JuPyter server restarts, make sure you run any blocks that start with #REQUIRED again before any changes.

### Deallocate Slice

This content is placed at the top to avoid mistakes from people running through the notebook using Shift-Enter.
Note that `fablib` is not imported unless the first code block after this section is run. 

In [ ]:
slice_name="NACgroupKey-" + fablib.get_bastion_username()
slice = fablib.get_slice(name=slice_name)
fablib.delete_slice(slice_name)

### Initial Setup (from fabric-ndn)

The bastion key and slice key must be in your `fabric_config/` JuPyter directory in order to properly operate the FabLib library on FABRIC.

Try to avoid skipping steps to prevent missing variables/imports.

In [ ]:
#REQUIRED

#Note: this format is used to avoid hardcoding user data either in the repo or in these code blocks.

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
conf = fablib.show_config()

### Topology Configuration (run to load variables)

In [ ]:
#at current the topology is intended to allocate on a single site

#configuring nodes for topology in *_conf variables
node_conf = [
    {"name": "dataowner", "cores": 2, "ram": 8, "disk": 20, "image": 'default_ubuntu_20', 'packages': ['apache2', 'python3', 'python3-pip', 'net-tools', 'iperf3', 'moreutils', 'pwntools']},
    {"name": "producer",  "cores": 2, "ram": 8, "disk": 20, "image": 'default_ubuntu_20', 'packages': ['apache2', 'python3', 'python3-pip', 'net-tools', 'iperf3', 'moreutils', 'pwntools']},
    {"name": "consumer",  "cores": 2, "ram": 8, "disk": 20, "image": 'default_ubuntu_20', 'packages': ['apache2', 'python3', 'python3-pip', 'net-tools', 'iperf3', 'moreutils', 'pwntools']},
    #{"name": "attacker",  "cores": 2, "ram": 8, "disk": 20, "image": 'default_ubuntu_20', 'packages': ['apache2', 'python3', 'python3-pip', 'net-tools', 'iperf3', 'moreutils', 'pwntools']},
    {"name": "router1",   "cores": 2, "ram": 8, "disk": 20, "image": 'default_ubuntu_20', 'packages': ['apache2', 'python3', 'python3-pip', 'net-tools', 'iperf3', 'moreutils', 'pwntools']}
]

net_conf = [
    {"name": "net_source", "subnet": "10.10.1.0/24", "nodes": [
        {"name": "dataowner", "addr": "10.10.1.30"},
        {"name": "producer",  "addr": "10.10.1.40"},
        {"name": "router1",   "addr": "10.10.1.10"}
    ]},
    {"name": "net_user", "subnet": "10.10.2.0/24", "nodes": [
        {"name": "consumer",  "addr": "10.10.2.50"},
        #{"name": "attacker",  "addr": "10.10.2.60"},
        {"name": "router1",   "addr": "10.10.2.10"}
    ]}
]

route_conf = [
    {"addr": "10.10.1.0/24", "gw": "10.10.1.10", "nodes": ["dataowner", "producer"]},
    {"addr": "10.10.2.0/24", "gw": "10.10.2.10", "nodes": ["consumer"]} #once implemented, potentially include    , "attacker"
]

#calculate config values for use in determining a valid site for allocation
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]), 'ram': sum([ n['ram'] for n in node_conf]) }

print("Configuration setup loaded.")

### Slice existence check and allocation

In [ ]:
#REQUIRED
slice_name="NACgroupKey-" + fablib.get_bastion_username()
try:
    slice = fablib.get_slice(name=slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

In [ ]:
## Run if Slice is not allocated

while True:
    site_name = fablib.get_random_site()
    #check if randomly chosen site has the resources to properly allocate
    if (    (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores'])
        and (fablib.resources.get_ram_available(site_name) > 1.2*exp_conf['ram'])
        #and (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic'])
       ):
        break

for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

fablib.show_site(site_name)

In [ ]:
#wait for visible output from previous
#it may be helpful to spot-check the resources availble on the other node
slice.submit()

In [ ]:
#REQUIRED

#wait for allocation success
slice.get_state()
slice.wait_ssh(progress=True)
#NOTE: examine output of this and the above block carefully to ensure the slice has been properly built

In [ ]:
#Display access to nodes through ssh
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

### Extend Slice Lifetime by 3 Days

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 3 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

### Configure resources within nodes

#### Step 1: Update Nodes, Install Useful Packages, Load Scripts

The node update and install thread will run in background; however, if it is not completed, some later parts may encounter issues.

In [ ]:
#GENERAL CONFIG

for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

from ipaddress import ip_address, IPv4Address, IPv4Network
for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]

for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
#Python library installations

#Once the above is done, use this to check complete Python 3 library installations
#It will produce a lot of text; if running pip does not say cannot find pip, it is working
for node in slice.get_nodes():
    #node.execute_thread("sudo apt install -y python3-pip; pip install cryptography; pip install rsa")
    node.execute("sudo apt install -y python3-pip; pip install cryptography; pip install rsa")

In [ ]:
#NODE SETUP
#Rerun this after any changes to scripts or config to ensure all pieces are in place for testing.
#Ensure the previous installs are done.

print("Loading scripts and content to nodes.")

#DATA OWNER
node = slice.get_node("dataowner")
try:
    result = node.upload_file('demoScripts/datowner.py','datowner.py')
    result = node.upload_file('demoScripts/datowner2.py','datowner2.py')
    print("Script loaded to data owner.")
    result = node.execute_thread('mkdir keySegments')
except Exception as e:
    print("Failed on node_%s for the following reason:"%(node.get_name()))
    print(e.message)
    
#PRODUCER
node = slice.get_node("producer")
try:
    result = node.upload_file('demoScripts/producer.py','producer.py')
    print("Script loaded to producer.")
    result = node.upload_file('contentExample/content.txt', 'content.txt')
    print("Content stored on producer.")
except Exception as e:
    print("Failed on node_%s for the following reason:"%(node.get_name()))
    print(e.message)

#NDN ROUTER
#node = slice.get_node("router1")
#try:
#    result = node.upload_file('demoScripts/ndnroute.py','ndnroute.py')
#    print("Script loaded to router.")
#except Exception as e:
#    print("Failed on node_%s for the following reason:"%(node.get_name()))
#    print(e.message)

#CONSUMER
node = slice.get_node("consumer")
try:
    result = node.upload_file('demoScripts/consumer.py','consumer.py')
    result = node.upload_file('demoScripts/consumer2.py','consumer2.py')
    print("Script loaded to consumer.")
except Exception as e:
    print("Failed on node_%s for the following reason:"%(node.get_name()))
    print(e.message)

#ENDING
print("Procedure over.")

#### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
#Draw topology for visibility
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes) + 5, len(nodes) + 1))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray', font_size=8, rotate=False);

#### Step 2: Run the following blocks to perform demo

Note: the demoCache folder can be checked for router contents between blocks.
In some cases, there shouldn't be anything.

(The first code block is a reset that will not cause problems even if the demo blocks haven't been used yet.)

In [ ]:
#RESET BLOCK

!rm -f demoCache/contentEncrypted
!rm -f demoCache/encryptedContentKey
!rm -f decryptedContent.txt
node = slice.get_node("dataowner")
node.execute("rm -f contentEncrypted; rm -f encryptedContentKey")
node = slice.get_node("consumer")
node.execute("rm -rf keySegments/; rm -f consumerPrivateKey; rm -f consumerPublicKey")
node = slice.get_node("producer")
node.execute("rm -f contentEncrypted; rm -f encryptedContentKey; rm -f groupEncKey")

In [ ]:
#DATA OWNER generates Group Key Pair
node = slice.get_node("dataowner")
result = node.execute("python3 datowner.py; ls -l")

In [ ]:
#PRODUCER retrieves Group Encryption Key

#We'll skip the request from Producer given the lack of interactibility using sockets and ports
node = slice.get_node("dataowner")
node.download_file("demoCache/groupEncKey","groupEncKey")
#Forward to Producer
node = slice.get_node("producer")
node.upload_file("demoCache/groupEncKey","groupEncKey")
#Remove from router cache
!rm demoCache/groupEncKey

#NOTE how router cache (demoCache directory) should not retain this interaction. In a real-world scenario, this would require an additional asymmetric exchange

In [ ]:
#Producer performs Content Encryption and Content Key Encryption
node = slice.get_node("producer")
result = node.execute("python3 producer.py; ls -l")

In [ ]:
#Producer provides content and content key (encrypted) by cacheing in Router (simulated using demoCache/)
node = slice.get_node("producer")
node.download_file("demoCache/encryptedContentKey","encryptedContentKey")
node.download_file("demoCache/contentEncrypted","contentEncrypted")

In [ ]:
#Consumer advertises own public key
node = slice.get_node("consumer")
result = node.execute("python3 consumer2.py; ls -l")

#We skip simulation of public key request

#Data Owner retrieves public key
node.download_file("demoCache/consumerPublicKey","consumerPublicKey")
node = slice.get_node("dataowner")
node.upload_file("demoCache/consumerPublicKey","consumerPublicKey")
!rm demoCache/consumerPublicKey

In [ ]:
#Data Owner encrypts Group decryption key using consumer public key
node = slice.get_node("dataowner")
result = node.execute("python3 datowner2.py; ls -l keySegments")

#Data Owner sends encrypted Group decryption key to consumer (simulated by transfer)
node.download_directory("demoCache", "keySegments")
node = slice.get_node("consumer")
node.upload_directory("demoCache/keySegments", "keySegments")
!rm -rf demoCache/keySegments
result = node.execute('mv keySegments/keySegments/* keySegments; rm -rf keySegments/keySegments')

In [ ]:
#Consumer retrieves encrypted content and content key
node = slice.get_node("consumer")
node.upload_file("demoCache/contentEncrypted","contentEncrypted")
node.upload_file("demoCache/encryptedContentKey","encryptedContentKey")

#Decrypt Group decryption key using consumer private key (reconstitute from segments)
result = node.execute("python3 consumer.py; ls -l")
node.download_file("decryptedContent.txt", "decryptedContent.txt")

In [ ]:
#Display final decryption output in JuPyter server after retrieving from consumer node

!cat decryptedContent.txt

#### And that's the demo!

If you saw the full decrypted message displayed using decryptedContent.txt, you can compare it to `contentExample/content.txt` to see it worked.

Return to the start of this notebook to deallocate slice.